https://discuss.pytorch.org/t/solved-concatenate-time-distributed-cnn-with-lstm/15435

In [4]:
# pytorch mnist cnn + lstm

from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import numpy as np

# Training settings
# parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
# parser.add_argument('--batch-size', type=int, default=64, metavar='N',
#                     help='input batch size for training (default: 64)')
# parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
#                     help='input batch size for testing (default: 1000)')
# parser.add_argument('--epochs', type=int, default=10, metavar='N',
#                     help='number of epochs to train (default: 10)')
# parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
#                     help='learning rate (default: 0.01)')
# parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
#                     help='SGD momentum (default: 0.5)')
# parser.add_argument('--no-cuda', action='store_true', default=False,
#                     help='disables CUDA training')
# parser.add_argument('--seed', type=int, default=1, metavar='S',
#                     help='random seed (default: 1)')
# parser.add_argument('--log-interval', type=int, default=10, metavar='N',
#                     help='how many batches to wait before logging training status')
# args = parser.parse_args()


class Args:
    def __init__(self):
        self.cuda = True
        self.no_cuda = False
        self.seed = 1
        self.batch_size = 50
        self.test_batch_size = 1000
        self.epochs = 10
        self.lr = 0.01
        self.momentum = 0.5
        self.log_interval = 10


args = Args()

args.cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        '../data',
        train=True,
        download=True,
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307, ), (0.3081, ))
        ])),
    batch_size=args.batch_size,
    shuffle=True,
    **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        '../data',
        train=False,
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307, ), (0.3081, ))
        ])),
    batch_size=args.test_batch_size,
    shuffle=True,
    **kwargs)

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        #x = F.relu(self.fc1(x))
        #x = F.dropout(x, training=self.training)
        #x = self.fc2(x)
        #return F.log_softmax(x, dim=1)
        return x


class Combine(nn.Module):
    def __init__(self):
        super(Combine, self).__init__()
        self.cnn = CNN()
        self.rnn = nn.LSTM(
            input_size=320, 
            hidden_size=64, 
            num_layers=1,
            batch_first=True)
        self.linear = nn.Linear(64,10)

    def forward(self, x):
        batch_size, timesteps, C, H, W = x.size()
        c_in = x.view(batch_size * timesteps, C, H, W)
        c_out = self.cnn(c_in)
        r_in = c_out.view(batch_size, timesteps, -1)
        r_out, (h_n, h_c) = self.rnn(r_in)
        r_out2 = self.linear(r_out[:, -1, :])
        
        return F.log_softmax(r_out2, dim=1)


model = Combine()
if args.cuda:
    model.cuda()

optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)


def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        
        data = np.expand_dims(data, axis=1)
        data = torch.FloatTensor(data)
        if args.cuda:
            data, target = data.cuda(), target.cuda()
            

        
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data))


def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        
        data = np.expand_dims(data, axis=1)
        data = torch.FloatTensor(data)
        print(target.size)
        
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(
            output, target, size_average=False).data  # sum up batch loss
        pred = output.data.max(
            1, keepdim=True)[1]  # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(
        '\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))


for epoch in range(1, args.epochs + 1):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.299300
Train Epoch: 1 [500/60000 (1%)]	Loss: 2.298595
Train Epoch: 1 [1000/60000 (2%)]	Loss: 2.288499
Train Epoch: 1 [1500/60000 (2%)]	Loss: 2.281582
Train Epoch: 1 [2000/60000 (3%)]	Loss: 2.303889
Train Epoch: 1 [2500/60000 (4%)]	Loss: 2.277070
Train Epoch: 1 [3000/60000 (5%)]	Loss: 2.271023
Train Epoch: 1 [3500/60000 (6%)]	Loss: 2.261115
Train Epoch: 1 [4000/60000 (7%)]	Loss: 2.246800
Train Epoch: 1 [4500/60000 (8%)]	Loss: 2.235594
Train Epoch: 1 [5000/60000 (8%)]	Loss: 2.232541
Train Epoch: 1 [5500/60000 (9%)]	Loss: 2.252884
Train Epoch: 1 [6000/60000 (10%)]	Loss: 2.220942
Train Epoch: 1 [6500/60000 (11%)]	Loss: 2.245444
Train Epoch: 1 [7000/60000 (12%)]	Loss: 2.216057
Train Epoch: 1 [7500/60000 (12%)]	Loss: 2.184268
Train Epoch: 1 [8000/60000 (13%)]	Loss: 2.185084
Train Epoch: 1 [8500/60000 (14%)]	Loss: 2.170418
Train Epoch: 1 [9000/60000 (15%)]	Loss: 2.095970
Train Epoch: 1 [9500/60000 (16%)]	Loss: 2.083909
Train Epoch: 1 [10000/60000 (17%)]	L

/cs/home/ybk1/python/lib/python3.7/site-packages/ipykernel_launcher.py:167: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


<built-in method size of Tensor object at 0x7f0b34090d70>
<built-in method size of Tensor object at 0x7f0b34090d70>
<built-in method size of Tensor object at 0x7f0b34090b90>
<built-in method size of Tensor object at 0x7f0ab552f6e0>
<built-in method size of Tensor object at 0x7f0ab552f6e0>
<built-in method size of Tensor object at 0x7f0ab552f2d0>
<built-in method size of Tensor object at 0x7f0ab65a3320>
<built-in method size of Tensor object at 0x7f0ab65a3320>

Test set: Average loss: 0.3561, Accuracy: 9065/10000 (91%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.759084
Train Epoch: 2 [500/60000 (1%)]	Loss: 0.508572
Train Epoch: 2 [1000/60000 (2%)]	Loss: 0.397960
Train Epoch: 2 [1500/60000 (2%)]	Loss: 0.409788
Train Epoch: 2 [2000/60000 (3%)]	Loss: 0.553995
Train Epoch: 2 [2500/60000 (4%)]	Loss: 0.431408
Train Epoch: 2 [3000/60000 (5%)]	Loss: 0.370855
Train Epoch: 2 [3500/60000 (6%)]	Loss: 0.312749
Train Epoch: 2 [4000/60000 (7%)]	Loss: 0.505629
Train Epoch: 2 [4500/60000 (8%)]	Loss: 0.532218